# TMDB 5000 EDA 

In [1]:
from dataset import TMDBDataset
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

## Credits 
### Cast

In [2]:
credits = pd.read_csv("tmp/raw/tmdb_5000_credits.csv")

In [3]:
actor_counts = {}
actor_names = {}
# Iterate over each row in the DataFrame
for index, row in credits.iterrows():
    cast_list = json.loads(row['cast'])  # Parse the JSON cast list
    actors = [(entry['id'], entry['name']) for entry in cast_list]  # Extract actor names
    
    # Update actor counts dictionary
    for actor_id, actor_name in actors:
        actor_counts[actor_id] = actor_counts.get(actor_id, 0) + 1
        actor_names[actor_id] = actor_name


In [4]:
actor_movies = pd.DataFrame({'actor_id': list(actor_counts.keys()), 'movie_count': list(actor_counts.values())})
actor_movies['actor_name'] = actor_movies['actor_id'].apply(lambda x: actor_names[x])
actor_movies.sort_values(by='movie_count', ascending=False)

,actor_id,movie_count,actor_name
541,2231,67,Samuel L. Jackson
8260,380,57,Robert De Niro
5427,62,51,Bruce Willis
4008,1892,48,Matt Damon
205,192,46,Morgan Freeman
...,...,...,...
25047,12820,1,Belinda Mayne
25048,1220237,1,Todd Carty
25050,1747343,1,Bronco McLoughlin
25052,1747344,1,Gerard Naprous


In [5]:
actor_movies["movie_count"].value_counts(normalize=True).head(10)

1     0.713105
2     0.135396
3     0.053931
4     0.028413
5     0.015461
6     0.011083
7     0.007419
8     0.005697
9     0.005203
10    0.003810
Name: movie_count, dtype: float64

More than 70% of actors appear in only one movie in the dataset. We will limit our analysis to actors who have appeared in at least 2 movies. It will simplify our analysis and will also make it more relevant.

In [6]:
actors_multiple_films = [actor_id for actor_id, count in actor_counts.items() if count > 1]

In [7]:
credits['cast'] = credits['cast'].apply(lambda x: json.dumps([entry for entry in json.loads(x) if entry['id'] in actors_multiple_films]))

### Crew

In [8]:
crew_counts = {}
crew_names = {}
# Iterate over each row in the DataFrame
for index, row in credits.iterrows():
    crew_list = json.loads(row['crew'])  # Parse the JSON cast list
    crew_members = [(entry['id'], entry['name']) for entry in crew_list]  # Extract actor names
    
    # Update actor counts dictionary
    for person_id, person_name in crew_members:
        crew_counts[person_id] = crew_counts.get(person_id, 0) + 1
        crew_names[person_id] = person_name


In [9]:
crew_movies = pd.DataFrame({'person_id': list(crew_counts.keys()), 'movie_count': list(crew_counts.values())})
crew_movies['person_name'] = crew_movies['person_id'].apply(lambda x: crew_names[x])
crew_movies.sort_values(by='movie_count', ascending=False)

,person_id,movie_count,person_name
10955,2294,104,Robert Rodriguez
1210,488,84,Steven Spielberg
6578,2952,83,Avy Kaufman
3440,5914,82,Mary Vernieu
3519,3965,75,Deborah Aquila
...,...,...,...
30721,1720844,1,Phil Pham
30720,1720842,1,Curt Frisk
30719,1720840,1,Beth Ostermann
30718,1720839,1,Mario Dumont


In [10]:
crew_movies["movie_count"].value_counts(normalize=True).head(10)

1     0.589638
2     0.169367
3     0.077262
4     0.044228
5     0.029044
6     0.019646
7     0.013841
8     0.010646
9     0.009133
10    0.007261
Name: movie_count, dtype: float64

Almost 59% of crew members worked on only one movie in the dataset. Similarly to what we did for the cast, we will limit our analysis to crew members who have worked on at least 2 movies.

In [11]:
crew_multiple_films = [person_id for person_id, count in crew_counts.items() if count > 1]

In [12]:
credits['crew'] = credits['crew'].apply(lambda x: json.dumps([entry for entry in json.loads(x) if entry['id'] in crew_multiple_films]))

## Movies

In [42]:
movies = pd.read_csv("tmp/raw/tmdb_5000_movies.csv")

### Revenue 
Removing the movies with no revenue

In [43]:
(movies["revenue"] == 0).sum(), (movies["revenue"] == 0).mean()

(1427, 0.29710597543202166)

There are 1427 movies in the dataset with revenue equal to 0 (about 30% of all movies). We will remove them from the dataset as based on our research, these appear to be errors in the data or missing values are labeled as such. This also allows us to reduce the size of the data. 

In [44]:
movies_to_keep = [movie["id"] for i, movie in movies.iterrows() if movie["revenue"] > 0]

In [45]:
movies = movies[movies["id"].isin(movies_to_keep)].reset_index(drop=True)

In [28]:
credits = credits[credits["movie_id"].isin(movies_to_keep)].reset_index(drop=True)

### Feature engineering

In [48]:
columns_to_drop = ["homepage", "original_title", "status"]
movies = movies.drop(columns_to_drop, axis=1)

In [46]:
movies["is_english"] = movies["original_language"].apply(lambda x: int(x == "en"))
movies.drop("original_language", axis=1, inplace=True)

In [68]:
movies["release_year"] = movies["release_date"].str.slice(0, 4).astype(int)
movies["release_month"] = movies["release_date"].str.slice(5, 7).astype(int)
movies.drop("release_date", axis=1, inplace=True)

Potential features to extract: 
- one hot encoding for genres
- one hot encoding for production companies
- one hot encoding for languages 

We can select only the most popular ones.

In [77]:
genre_counts = {}
for row in movies["genres"]:
    genres = json.loads(row)
    for genre in genres:
        genre_name = genre['name']
        genre_counts[genre_name] = genre_counts.get(genre_name, 0) + 1

In [37]:
genre_counts

{'Action': 940,
 'Adventure': 675,
 'Fantasy': 351,
 'Science Fiction': 439,
 'Crime': 537,
 'Drama': 1527,
 'Thriller': 959,
 'Animation': 191,
 'Family': 382,
 'Western': 61,
 'Comedy': 1174,
 'Romance': 604,
 'Horror': 344,
 'Mystery': 268,
 'History': 150,
 'War': 122,
 'Music': 121,
 'Documentary': 47,
 'Foreign': 6}

In [53]:
spoken_languages = {}
for row in movies["spoken_languages"]:
    languages = json.loads(row)
    for lan in languages:
        lan_iso = lan['iso_639_1']
        spoken_languages[lan_iso] = spoken_languages.get(lan_iso, 0) + 1

In [54]:
spoken_languages

{'en': 3262,
 'es': 281,
 'fr': 316,
 'it': 146,
 'de': 184,
 'tr': 16,
 'el': 20,
 'zh': 82,
 'th': 35,
 'is': 5,
 'ru': 153,
 'sv': 16,
 'ro': 9,
 'ja': 83,
 'la': 43,
 'hi': 27,
 'pt': 49,
 'bo': 3,
 'fa': 7,
 'ur': 13,
 'ar': 56,
 'sa': 1,
 'gd': 5,
 'cs': 29,
 'cn': 35,
 'ko': 25,
 'no': 8,
 'ta': 2,
 'nv': 1,
 'he': 25,
 'da': 8,
 'nl': 7,
 'af': 6,
 'ga': 5,
 'so': 2,
 'fi': 3,
 'bg': 3,
 'yi': 6,
 'vi': 15,
 'hu': 24,
 'uk': 9,
 'eo': 2,
 'pl': 29,
 'et': 2,
 'sq': 4,
 'hr': 3,
 'tl': 1,
 'sh': 2,
 'sk': 1,
 'ml': 2,
 'te': 1,
 'cy': 2,
 'hy': 3,
 'wo': 1,
 'xh': 3,
 'km': 1,
 'sw': 5,
 'ny': 1,
 'st': 1,
 'zu': 3,
 'kw': 1,
 'sr': 4,
 'si': 1,
 'ne': 1,
 'ps': 1,
 'kk': 1,
 'mn': 1,
 'xx': 2,
 'gl': 1,
 'bs': 1,
 'ka': 1,
 'bn': 1,
 'mi': 2,
 'to': 1,
 'ca': 1,
 'pa': 1,
 'br': 1,
 'id': 2}

In [71]:
production_companies = {}
for row in movies["production_companies"]:
    companies = json.loads(row)
    for c in companies:
        c_name = c['name']
        production_companies[c_name] = production_companies.get(c_name, 0) + 1

In [76]:
pd.DataFrame({'company': list(production_companies.keys()), 'count': list(production_companies.values())}).sort_values(by='count', ascending=False).head(20)

,company,count
11,Warner Bros.,285
52,Universal Pictures,276
38,Paramount Pictures,251
1,Twentieth Century Fox Film Corporation,205
7,Columbia Pictures,168
46,New Line Cinema,144
4,Walt Disney Pictures,100
54,Relativity Media,100
49,Metro-Goldwyn-Mayer (MGM),96
193,Touchstone Pictures,95


In [83]:
credits.to_csv("tmp/raw/tmdb_5000_credits_processed.csv", index=False)
movies.to_csv("tmp/raw/tmdb_5000_movies_processed.csv", index=False)